![Pandas logo](img/pandas.svg)

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.training import *

# Groupby Operations

A powerful way of analyzing data in DataFrames is to use the `.groupby()` method, and perform operations on the resulting `DataFrameGroupBy` object.

For several examples, we will load a slightly enhanced version of the Olympic medal data we worked with before.  In this case, the medal categories have been combined in one DataFrame, and some additional categorical data has been added.

We are not entirely pleased with the continent classifications, with the Carribean called North America, for example, but most countries are classified.  This data spans years covering a number of changes in names and territorial boundaries of countries, so it is messy in much the way most data is.

In [ ]:
medals = pd.read_csv('data/olympic-medals.csv', index_col='Name')
medals

Let us take a look at what continents and "levels" exist for this purpose.

In [ ]:
print(medals.Continent.unique())
print(medals.Level.unique())

## What Do Like Things Have in Common?

The basic question grouping answers—in Pandas and in SQL whence the concept is borrowed—is what properties pertain to items that are categorically similar.  Grouping does not really make sense for continuous quantities; but it often does for scales divided by thresholds (as we do with 'Level' here).

In [ ]:
group = medals.groupby('Level')
print(group)
group[["Gold", "Silver", "Bronze", "Total"]].mean()

We can see the different "typical of achievement group" numbers of medals.  This would look better if the index was sorted more descriptively.

In [ ]:
(medals
     .groupby('Level')
     [["Gold", "Silver", "Bronze", "Total"]]
     .mean()
     .sort_values('Total', ascending=False)
)

How many nations fall in each group?

In [ ]:
medals.groupby('Level').Total.count().sort_values()

The different achievement levels for nations was set by thresholds.  Let us try to reconstruct what those thresholds were.

In [ ]:
medals.groupby('Level').Total.min()

In [ ]:
medals.groupby('Level').Total.max()

## Multiple Aggregations

We got a sense of the thresholds by looking at minimum and maximum, but the presentation is not that convenient.  Moreover, it is less easy to work with later as separate DataFrames or Series representing a single aggregation.

In [ ]:
(medals
     .groupby('Level')
     [["Gold", "Silver", "Bronze", "Total"]]
     .agg(['min', 'max', 'mean'])
)

The thresholds were set, as it happens, using total medals rather than one category. We can focus on that.

In [ ]:
# Notice that some common aggregations have names
# ... but we can also pass in an arbitrary ufunc
(medals
     .groupby('Level')
     .Total
     .agg(['min', 'max', 'mean', np.std])
     .sort_values('min')
)

## Nested Grouping

We might be interested in the combination of multiple categorical fields.  Notice that the levels are based on total medals.

In [ ]:
with show_all_rows():
    print(medals.groupby(['Level', 'Continent']).max())

We can combine several aggregations, and include a novel one: `list`.  That is, simply the Python function list.  Let us look at all the different Gold medal counts per each country within each nested group.  Also we simultaenously find the median of the group and a count of how many countries make up each group.

In [ ]:
medals.groupby(['Level', 'Continent']).Gold.agg(['count', 'median', 'max', list])

Which country within each nested group was the one that actually won that highest number of Gold medals?  `.idxmax()` is an interesting aggregation that gives you the actual index of the item matching the maximum.  More advanced users of NumPy will know `.argmax()` and `.argsort()` which work similarly.

In [ ]:
medals.groupby(['Level', 'Continent']).Gold.idxmax()

In [ ]:
medals.groupby(['Level', 'Continent']).Gold.agg(['idxmax', 'max'])

## Group By Derived Feature

The categorical distinction Pandas can group by does not need to be an actual column.  It could potentially be something that can be calculated from a column, or even from the index.

This is a bit silly as a practical need, but let us group countries by the last letter of their name, just to illustrate.  There were 6545 medals won by countries whose name ends in 'a'.

In [ ]:
medals.groupby(medals.index.str[-1]).Total.sum()

A bit more plausibly, and similar to the operation done in advance to derive `Level`, we can look at the logarithm of the Gold medal count to get a modest number of categories.  We add one to the count because we cannot take the log of zero.

In [ ]:
grouper = medals.groupby(np.log(medals.Gold + 1).astype(int))
grouper.Gold.agg(["count", "min", "max"])

# Exercises

For these exercises, we will again work with our NOAA temperature dataset in the large version.  We will derive some categorical features.  The last section of this module that looked at derived features will be relevant.

Just as a suggestion, you may find it easier to derive a feature and place that derived value in a new column to work with.  Generally, Pandas will always give you ways to achieve the goal without that intermediate step.  However, the "fluent style" of Pandas that chain many methods can become confusing quickly to beginners.  Saving intermediate results in new objects or in new columns is fine.

In [ ]:
# For exercises, we probably want to look at more rows in results
pd.options.display.max_rows = 50
url = ("https://bitbucket.org/davidmertz/sample-data/raw/"
       "61872271984f66e3094c367cf90dfc4875a22e8d/NOAA-2019-partial.csv.gz")
temperatures = pd.read_csv(url)
temperatures['DATE'] = pd.to_datetime(temperatures.DATE, format="%Y-%m-%d")

In [ ]:
temperatures.columns

You may have noticed that the data in this dataset is predominantly from stations in northern latitudes.  That does not reflect a limitation of NOAA's data collection, but simply the numbering system they use for download files.  In putting together the sample data for this course, we only downloaded files whose names begin with the numeral '0' (then aggregated them for the dataset you have worked with).

### Clean the Data

First step, remove all the latitudes south of 40°N.  This is a relatively small number of rows, as described above.

In [ ]:
# Remove data outside northern circumference
...

### Create Categorical Data

We would like to divide the data between 40°N and 90°N into latitude bands of 10 degrees.  Save those band memberships in a new column called `Latitude_Band`.  You may use either a string or a number to represent the bands, the point is simply to have a small number of them rather than a continuous range.

In [ ]:
# Make latitude categorical
...

We would also like to create a new categorical column called `Month` that represents which month of the year a measurement was in.  Ideally, store the string name of the month in that column, but an integer is acceptable.

In [ ]:
# Make month categorical
...

One more categorical to go.  We would like to divide elevation into bands, similarly to what you did with latitude.  However, elevations are clustered at lower numbers, so we wish to create `Elevation_Band` based on the log base-10 of the raw elevation. I.e. it will be in 5 bands with minima of 0, 1, 10, 100, 1000.

**Note**: Actually there is a trick here! A few of the stations are below sea level and have negative elevations.  Let us make those be a separate category also.

## Group and Aggregate

We would like to look at the following:

* Mean temperature, windspeed, and precipiation, simply by month.
* Mean and maximum temperature, windspeed, and precipitation by month and latitude band.
* Think also about what you read at [Global Surface Summary of the Day](https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00516) in deciding the most appropriate data fields to use.
* Minimum and maximum temperature and atmospheric pressure by month, latitude band, and elevation band.
* The total variability of temperatures within each particular month, latitude band, and elevation band group.  Again think about choosing the best featuers to answer this.

In [ ]:
# Aggregate numerous quanities by various categoricals
...